# Pacific Hurricanes 1975 extractor
Web scraper for the 1975 Pacific hurricane season Wikipedia page

![](1975_Pacific_hurricane_season_summary_map_new.png)

### 1. Importing Required Libraries and Functions
Here, we import the necessary libraries, including the custom `functions.py` file for utility functions.

In [1]:
# Import custom utility functions from the 'functions.py' file as 'f'
import functions as f

### 2. Scraping the Wikipedia Page for Pacific Hurricanes 1975
We fetch the Wikipedia page for the 1975 Pacific hurricane season and parse its HTML content using BeautifulSoup.


In [2]:
# URL for the 1975 Pacific hurricane season Wikipedia page
url = 'https://en.wikipedia.org/wiki/1975_Pacific_hurricane_season'

# Call function to get and parse the HTML content of the URL
soup = f.get_url_content(url)

Successfully fetched the requested page.


### 3. Extracting Hurricane and Storm Names
We search the parsed HTML for hurricane and storm names in the appropriate `<div>` elements.

In [3]:
# Find all <div> elements containing class 'mw-heading mw-heading3' (hurricane/storm names)
div_element = soup.find_all('div', {'class': 'mw-heading mw-heading3'})

# Initialize a list to hold storm/hurricane names
hurricane_storm_name = []

# Loop through each div element to extract storm names
for h3 in div_element:
    striped_h3 = h3.find('h3').text.strip()  # Extract and clean text of the storm name
    hurricane_storm_name.append(striped_h3)  # Append the cleaned name to the list

### 4. Extracting Start and End Dates for Each Hurricane/Storm
We locate the tables that contain the start and end dates for each hurricane or storm and store them in lists.


In [4]:
# Find all <table> elements with class 'infobox' (information boxes)
table_element = soup.find_all('table', {'class': 'infobox'})

# Filter tables that have exactly 'infobox' class (ignore other table types)
exact_class_tables = [table for table in table_element if table['class'] == ['infobox']]

# Initialize lists to hold start and end dates
start_date = []
end_date = []

# Loop through the filtered infobox tables to extract storm dates
for td in exact_class_tables:
    # Extract the start and end dates of the storm (strip extra characters like '\xa0–')
    striped_date = td.find('td', {'class': 'infobox-data'}).text.strip().split('\xa0– ')
    start_date.append(striped_date[0])  # Append start date
    end_date.append(striped_date[1])    # Append end date

### 5. Extracting Descriptions for Each Hurricane/Storm
We now scrape the paragraphs of information related to each storm and store them in a list for further analysis.


In [5]:
# Initialize a list to hold storm/hurricane descriptions
info_text = []

# Find all <div> elements with class 'mw-heading mw-heading3' for each storm's description
all_h3 = soup.find_all('div', {'class': 'mw-heading mw-heading3'})

# Loop through each heading and extract its corresponding paragraphs
for h3 in all_h3:
    content_between_h3 = []
    
    # Loop through sibling tags after each <div> tag until another <div> tag is found
    for sibling in h3.find_next_siblings(recursive=False):
        # Break the loop if another heading <div> is found
        if sibling.name == 'div' and sibling.attrs == {'class': ['mw-heading', 'mw-heading3']}:
            break
        # Collect text from <p> (paragraphs)
        if sibling.name == 'p':
            text = sibling.text.strip()  # Clean and extract paragraph text
            content_between_h3.append(text)
    
    # Join paragraphs into a single string
    info_text.append(''.join(content_between_h3))

# Remove the last element from the info_text list
info_text = info_text[:-1]

### 6. Using GPT to Extract Structured Information from the Descriptions
We use the OpenAI GPT model to extract the number of deaths and areas affected by each hurricane/storm.


In [6]:
# Initialize a list to hold extracted information and token counts
result = []
tokens_used = 0
prompt_tokens = 0
completion_tokens = 0

# Loop through each hurricane description and extract information using the GPT function
for text in info_text:
    response = f.extract_info_from_text(text)  # Extract data using OpenAI GPT
    text_response = response.choices[0].message.content  # Get GPT response
    result.append(text_response)  # Append extracted data to the result list
    
    # Track usage statistics for OpenAI API tokens
    tokens_used += response.usage.total_tokens
    prompt_tokens += response.usage.prompt_tokens
    completion_tokens += response.usage.completion_tokens

### 7. Processing the Extracted Information
We process the GPT-extracted data, converting it into a usable format (Python dictionaries) and extract relevant details such as the number of deaths and areas affected.


In [7]:
# Import JSON to handle the response data
import json

# Initialize lists to hold extracted deaths and affected areas data
deaths = []
affected_areas = []

# Loop through each GPT response and convert it to a Python dictionary
for item in result:
    # Replace single quotes with double quotes to ensure valid JSON format
    item_n = item.replace("'", '"')
    info = json.loads(item_n)  # Convert string to dictionary
    
    # Extract deaths and affected areas from the dictionary
    deaths.append(info['number_of_deaths'])
    affected_areas.append(info['areas_affected'])

### 8. Creating a DataFrame
We create a pandas DataFrame to organize the hurricane information, including storm names, dates, deaths, and affected areas.


In [8]:
# Import pandas to handle dataframes
import pandas as pd

# Create a dictionary with all columns (storm names, dates, deaths, areas)
columns = {
    'hurricane_storm_name': hurricane_storm_name[:-1],  # Exclude the last entry
    'date_start': start_date,
    'date_end': end_date,
    'number_of_deaths': deaths,
    'list_of_areas_affected': affected_areas
}

# Create a pandas DataFrame from the dictionary
df = pd.DataFrame(columns)

### 9. Formatting Dates and Areas
We apply additional formatting to the start and end dates and combine the list of affected areas into a single string for each storm.


In [9]:
# Apply the date conversion function to start and end dates
df['date_start'] = df['date_start'].apply(f.convert_date)
df['date_end'] = df['date_end'].apply(f.convert_date)

In [10]:
# Join the list of areas affected into a single string for each storm
df['list_of_areas_affected'] = df['list_of_areas_affected'].apply(lambda x: ', '.join(x))

In [11]:
df

,hurricane_storm_name,date_start,date_end,number_of_deaths,list_of_areas_affected
0,Hurricane Agatha,1975-06-02,1975-06-05,0,"Acapulco, Zihuatanejo, Tres Marias Islands, Sa..."
1,Tropical Storm Bridget,1975-06-28,1975-07-03,0,Baja California Peninsula
2,Hurricane Carlotta,1975-07-02,1975-07-11,0,Acapulco
3,Hurricane Denise,1975-07-05,1975-07-15,0,parts of Mexico
4,Tropical Storm Eleanor,1975-07-10,1975-07-12,0,"Acapulco, Manzanillo"
5,Tropical Storm Francene,1975-07-27,1975-07-30,0,"northwestward, west"
6,Tropical Storm Georgette,1975-08-11,1975-08-14,0,Cabo San Lucas
7,Tropical Storm Hilary,1975-08-13,1975-08-17,0,No areas were affected as the tropical storm d...
8,Hurricane Ilsa,1975-08-18,1975-08-26,0,"Gulf of Tehuantepec, Pacific Ocean, Atlantic O..."
9,Hurricane Jewel,1975-08-24,1975-08-31,0,"Acapulco, Mexico"


### 10. Exporting Data to CSV
The cleaned and formatted hurricane data is exported to a CSV file for further analysis.

In [12]:
# Export the DataFrame to a CSV file
df.to_csv('hurricanes_1975.csv', index=False)

### 11. Logging Token Usage
We log the total tokens used during the GPT API calls, including prompt and completion tokens.


In [13]:
# Write the token usage information to a log file
with open('log.txt', 'w') as file:
    file.write('tokens_used: ' + str(tokens_used) + '\n')
    file.write('prompt_tokens: ' + str(prompt_tokens) + '\n')
    file.write('completion_tokens: ' + str(completion_tokens))